# Predicting a 'no show' for a medical appointment based on historical data
This notebook uses a historical dataset from 2016 to predict someone not showing up for a medical appointment.
## Packages

In [98]:
import numpy as np
import pandas as pd

## Data input
Read in the dataset.

In [99]:
df = pd.read_csv('data/medical_no_show.csv')
print('Count of rows', str(df.shape[0]))
print('Count of Columns', str(df.shape[1]))
df.head()

Count of rows 110527
Count of Columns 14


,PatientId,AppointmentID,Gender,ScheduledDay,AppointmentDay,Age,Neighbourhood,Scholarship,Hipertension,Diabetes,Alcoholism,Handcap,SMS_received,No-show
0,2.987250e+13,5642903,F,2016-04-29T18:38:08Z,2016-04-29T00:00:00Z,62,JARDIM DA PENHA,0,1,0,0,0,0,No
1,5.589978e+14,5642503,M,2016-04-29T16:08:27Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,0,0,0,0,0,No
2,4.262962e+12,5642549,F,2016-04-29T16:19:04Z,2016-04-29T00:00:00Z,62,MATA DA PRAIA,0,0,0,0,0,0,No
3,8.679512e+11,5642828,F,2016-04-29T17:29:31Z,2016-04-29T00:00:00Z,8,PONTAL DE CAMBURI,0,0,0,0,0,0,No
4,8.841186e+12,5642494,F,2016-04-29T16:07:23Z,2016-04-29T00:00:00Z,56,JARDIM DA PENHA,0,1,1,0,0,0,No


In the next part we check for missing data.

In [100]:
df.isnull().any().any()

False

We also check the dtypes for each of the columns.

In [101]:
df.dtypes

PatientId         float64
AppointmentID       int64
Gender             object
ScheduledDay       object
AppointmentDay     object
Age                 int64
Neighbourhood      object
Scholarship         int64
Hipertension        int64
Diabetes            int64
Alcoholism          int64
Handcap             int64
SMS_received        int64
No-show            object
dtype: object

Furthermore, we check how many unique values there are for each column.

In [102]:
for i in df.columns:
    print(i+":",len(df[i].unique()))

PatientId: 62299
AppointmentID: 110527
Gender: 2
ScheduledDay: 103549
AppointmentDay: 27
Age: 104
Neighbourhood: 81
Scholarship: 2
Hipertension: 2
Diabetes: 2
Alcoholism: 2
Handcap: 5
SMS_received: 2
No-show: 2


## Data Cleaning
First all column names are converted lowercase to achieve consistency.

In [103]:
df.columns = df.columns.str.lower().str.strip()

`patientid` needs to be converted to `int`.  
`no-show` needs to be converted to `int`.  
`gender` needs to be converted to `int`.  

In [104]:
df['patientid'] = df['patientid'].astype('int64')
df['no-show'] = df['no-show'].map({'No':0, 'Yes':1})
df['gender'] = df['gender'].map({'F':0, 'M':1})

`neighbourhood` is converted using one hot encoding.

In [105]:
df = pd.get_dummies(df, columns = ['neighbourhood'])

A couple of features were added:
- `previous_app`: count how many previous appointments the patient has had (starting with 0)
- `previous_noshow`: percentage of missed appointments
- `apps_missed`: number of appointments missed

In [113]:
df['num_app'] = df.sort_values(by = ['patientid','scheduledday']).groupby(['patientid']).cumcount() + 1
df['noshow_pct'] = (df.sort_values(['patientid', 'scheduledday']).groupby(['patientid'])['no-show'].cumsum() / df['num_app'])
df['apps_missed'] = df.groupby('patientid')['no-show'].apply(lambda x: x.cumsum())

In [114]:
df[df['patientid'] == 838284762259].sort_values(by = ['patientid','scheduledday'])[['no-show', 'num_app', 'noshow_pct', 'apps_missed']]

,no-show,num_app,noshow_pct,apps_missed
1856,0,1,0.000000,0
3655,0,2,0.000000,0
11004,1,3,0.333333,1
9166,0,4,0.250000,0
16924,1,5,0.400000,2
9176,0,6,0.333333,0
3590,0,7,0.285714,0
3715,0,8,0.250000,0
3718,0,9,0.222222,0
3728,0,10,0.200000,0


In [110]:
df[(df['no-show'] == 1) & (df['num_app'] > 2)]

,patientid,appointmentid,gender,scheduledday,appointmentday,age,scholarship,hipertension,diabetes,alcoholism,...,neighbourhood_SOLON BORGES,neighbourhood_SÃO BENEDITO,neighbourhood_SÃO CRISTÓVÃO,neighbourhood_SÃO JOSÉ,neighbourhood_SÃO PEDRO,neighbourhood_TABUAZEIRO,neighbourhood_UNIVERSITÁRIO,neighbourhood_VILA RUBIM,num_app,noshow_pct
152,37976483781944,5629610,0,2016-04-27T13:46:37Z,2016-04-29T00:00:00Z,18,0,0,0,0,...,0,0,0,0,0,0,0,0,3,1.000000
251,653745118443,5640178,0,2016-04-29T10:13:22Z,2016-04-29T00:00:00Z,33,1,0,0,0,...,0,0,0,0,0,0,0,0,3,0.333333
668,56548277857,5599192,0,2016-04-19T08:35:26Z,2016-04-29T00:00:00Z,40,0,0,0,0,...,0,0,0,0,0,0,0,0,4,0.750000
848,343735171537732,5625977,1,2016-04-27T07:46:31Z,2016-04-29T00:00:00Z,43,0,0,0,0,...,0,0,0,0,0,1,0,0,3,0.666667
1004,236326746564753,5637240,1,2016-04-28T18:02:54Z,2016-04-29T00:00:00Z,46,0,1,0,0,...,0,0,0,0,0,0,0,0,3,1.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110420,2934141357952,5574038,0,2016-04-12T14:01:07Z,2016-06-06T00:00:00Z,41,0,0,0,0,...,0,0,0,0,0,0,0,0,4,0.250000
110430,49861634253456,5736999,0,2016-05-25T09:01:33Z,2016-06-01T00:00:00Z,57,0,0,0,0,...,0,0,0,0,0,0,0,0,4,0.750000
110492,645634214296344,5786741,1,2016-06-08T08:50:19Z,2016-06-08T00:00:00Z,33,0,1,0,0,...,0,0,0,0,0,0,0,0,3,0.666667
110496,85442954737999,5779046,0,2016-06-06T17:35:38Z,2016-06-08T00:00:00Z,37,0,1,0,0,...,0,0,0,0,0,0,0,0,3,1.000000
